In [89]:
path = '/Users/simonbian/大三下/CS470/project/combined.csv'
import pandas as pd
df = pd.read_csv(path)
def standardize_degree(degree_name):
    if 'phd' in degree_name.lower().replace('.', '').replace(' '):
        return 'Ph.D.'
    if 'master' in degree_name.lower():
        return 'Masters'
    # Add more conditions here to standardize other degree names
    return degree_name

# Iterate over each row to separate terms and degrees
for index, row in df.iterrows():
    value = row['Attribute_1']
    try:
        if value.startswith('Fall') or value.startswith('Spring'):
            df.at[index, 'Term'] = value
        else:
            standardized_degree = standardize_degree(value)
            df.at[index, 'Degree'] = standardized_degree
    except:
        pass


/var/folders/0p/bfk7s5314wzfkww172k0x4280000gn/T/ipykernel_73171/3265399693.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [91]:
df_small = df.copy()
df_small['GPA'] = df_small['GPA'].str.replace("GPA ", "")
df_small[['Status','Date']]=df_small['Status_Date'].str.split(' ', n=1, expand=True)
df_small[['Major', 'School']] = df_small['Major_School'].str.rsplit(',', 1, expand=True)
df_small['Date'] = df_small['Date'].str.replace(" on ", "", regex=False)
df_small.drop('Major_School', axis=1,inplace=True)
df_small.drop('Status_Date', axis=1,inplace=True)
import numpy as np
def extract_scores(row):
    if pd.isna(row):
        return np.nan, np.nan, np.nan  # Handle NA entries
    parts = row.split(';')
    gre_q, gre_v, gre_aw = np.nan, np.nan, np.nan
    for part in parts:
        if 'GRE ' in part and 'V' not in part and 'AW' not in part:
            gre_q = part.split()[-1]
        elif 'GRE V' in part:
            gre_v = part.split()[-1]
        elif 'GRE AW' in part:
            gre_aw = part.split()[-1]
    return gre_q, gre_v, gre_aw
df_small['GRE_Q'], df_small['GRE_V'], df_small['GRE_AW'] = zip(*df_small['GRE_Stats'].progress_apply(extract_scores))
df_small['GRE_Q'] = pd.to_numeric(df_small['GRE_Q'], errors='coerce')
df_small['GRE_V'] = pd.to_numeric(df_small['GRE_V'], errors='coerce')
df_small['GRE_AW'] = pd.to_numeric(df_small['GRE_AW'], errors='coerce')
df_small.drop('Attribute_1', axis=1,inplace=True)
df_small.drop('Attribute_2', axis=1,inplace=True)
df_small.drop('Attribute_3', axis=1,inplace=True)
df_small.drop('GRE_Stats', axis=1,inplace=True)

df_small['Date'] = pd.to_datetime(df_small['Date_Added'])
df_small.drop('Date_Added',axis=1,inplace=True)
import pandas as pd
import re

def extract_publications(text):
    if pd.isnull(text):
        return 0
    return len(re.findall(r'\bpub\b|\bpublication\b|\bpaper\b', text, flags=re.IGNORECASE))

# Function to extract RA/TA mentions
def extract_ra_ta(text):
    if pd.isnull(text):
        return 0
    return len(re.findall(r'\bRA\b|\bTA\b', text))

# Function to extract funding information
def extract_funding(text):
    if pd.isnull(text):
        return 0
    # Find amounts mentioned with $
    funds = re.findall(r'\$\d+', text)
    if funds:
        return int(re.sub(r'\$', '', funds[0]))
    # Check for keywords "fund" or "arship" for scholarships
    if re.search(r'\bfund\b|\barship\b', text, flags=re.IGNORECASE):
        return 1  # Indicating presence of funding mention without specific amount
    return 0
df_small['Publications'] = df_small['Description'].progress_apply(extract_publications)
df_small['RA_TA'] = df_small['Description'].progress_apply(extract_ra_ta)
df_small['Funding'] = df_small['Description'].progress_apply(extract_funding)
df_small.drop('Description', axis=1,inplace=True)
import fuzzywuzzy
from fuzzywuzzy import process
import pandas as pd
from tqdm import tqdm
tqdm.pandas()  # Prepare tqdm for pandas

schools = pd.read_csv('/Users/simonbian/Documents/GitHub/Data_Mining/4. Programs Analysis (Some simple histograms)/schools.csv')
# Assuming 'schools' and 'df' are your DataFrames
schools.drop('Description', axis=1,inplace=True)
schools.drop('In-state', axis=1,inplace=True)
schools.drop('Undergrad Enrollment',axis=1,inplace=True)
# Initialize a hashmap (dictionary) to store previously computed matches
matches_cache = {}

def find_most_similar_with_cache(row, choices, scorer, cache):
    # Check if the current row's value is already in the cache
    if row in cache:
        return cache[row]
    else:
        try:
            # If not in cache, find the most similar match and store it in the cache
            row = row.replace("Penn","Pennsylvania").replace("MIT",'Massachusetts Institute of Technology').replace('Berkeley','University of California--Berkeley').replace('UCLA','University of California--Los Angeles')
            row = row.replace('UCSB','University of California--Santa Barbara')
            row = row.replace('UCI','University of California--Irvine')
            row = row.replace('UC','University of California')
            row = row.replace("UCD",'University of California--Davis')
            row = row.replace('UCSD','University of California--San Diego')
            row = row.replace("SD",'San Diego')
            row = row.replace("SB",'--Santa Barbara')
            row = row.replace("Princeton","Princeton University")
            row = row.replace("Penn State University",'Pennsylvania State University--University Park')
            match, score = process.extractOne(row, choices, scorer=scorer)
            cache[row] = match
            return match
        except:
            return "None"

# Prepare the list of choices from schools DataFrame
choices = schools['Name'].tolist()

# Apply the function to each row in df['School'], now using a cache for optimization
df_small['School_Sim'] = df_small['School'].progress_apply(find_most_similar_with_cache, args=(choices, fuzzywuzzy.fuzz.token_sort_ratio, matches_cache))

# Proceed to merge the DataFrames based on the found most similar school names
df_small = pd.merge(schools, df_small, left_on='Name', right_on='School_Sim', how='left')
df_small.drop('Location',axis=1,inplace=True)# for the moment, we use rank only
df_small.drop('Tuition and fees',axis=1,inplace=True)
df_small.drop('Name',axis=1,inplace=True)
df_small.drop('School_Sim',axis=1,inplace=True)

/var/folders/0p/bfk7s5314wzfkww172k0x4280000gn/T/ipykernel_73171/3486463989.py:4: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df_small[['Major', 'School']] = df_small['Major_School'].str.rsplit(',', 1, expand=True)
100%|██████████████████████████████████| 847725/847725 [35:05<00:00, 402.60it/s]


In [92]:
mask = df_small.notna().sum(axis=1) > 1
# Filter out rows that do not meet the criteria
filtered_df =df_small[mask]
filtered_df.to_csv("maxi.csv")

In [104]:
filtered_df = pd.read_csv("/Users/simonbian/大三下/CS470/local_temp/maxi.csv")

In [108]:
filtered_df

,Unnamed: 0,Rank,GPA,Degree,Citizenship,Term,Status,Date,Major,School,GRE_Q,GRE_V,GRE_AW,Publications,RA_TA,Funding
0,0,1,3.99,PhD,International,Fall 2024,Accepted,2024-01-29,Ecology And Evolutionary Biology,Princeton University,170.0,159.0,4.5,0,0,0
1,1,1,NaN,PhD,International,Fall 2024,Interview,2024-01-28,East Asian Studies,Princeton University,NaN,NaN,NaN,0,0,0
2,2,1,NaN,PhD,American,Fall 2024,Accepted,2024-01-28,Politics,Princeton University,NaN,NaN,NaN,0,0,0
3,3,1,3.82,PhD,American,Fall 2024,Interview,2024-01-28,Sociocultural Anthropology,Princeton University,NaN,NaN,NaN,0,0,0
4,4,1,NaN,PhD,International,Fall 2024,Accepted,2024-01-27,Politics,Princeton University,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847719,847719,220,NaN,PhD,International,NaN,Accepted,2007-03-15,Physics,Wayne State University,NaN,NaN,NaN,0,0,0
847720,847720,220,NaN,PhD,International,NaN,Accepted,2006-04-18,Communication,Wayne State University,NaN,NaN,NaN,0,1,0
847721,847721,220,NaN,Masters,International,NaN,Accepted,2006-03-22,Sociology,Utah State University,NaN,NaN,NaN,0,0,0
847722,847722,220,NaN,PhD,American,NaN,Rejected,2006-02-21,Clinical Psychology,Wayne State University,NaN,NaN,NaN,0,0,0


In [111]:
ddf = filtered_df.copy()
ddf.dropna(subset=['Citizenship', 'Status'], inplace=True)
ddf['GPA'] = pd.to_numeric(ddf['GPA'], errors='coerce')

# Dropping rows where GPA is NaN after conversion (if any)
ddf.dropna(subset=['GPA'], inplace=True)
# Filtering applicants with GPA greater than 3.5
gpa_filtered_df = ddf[ddf['GPA'] > 3.5]
contingency_table = pd.crosstab(gpa_filtered_df['Citizenship'], gpa_filtered_df['Status'])

# Conducting Chi-square test to see if citizenship affects the acceptance results
chi2, p, dof, expected = chi2_contingency(contingency_table)


In [113]:
contingency_table

Status,Accepted,Interview,Other,Rejected,Wait
Citizenship,,,,,
American,53877,13210,2183,31399,7407
International,35273,5549,1868,27537,3202
